Surname Classification with RNN
===============================

In this example, we see surname classification in which character sequences (surnames) are classified to nationality of origin.

Inferring demographic information (like nationality) from publicly observable data has applications from product recommendations to ensuring fair outcomes for users across different demographics.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/TextPro2566/Surname'

/content/drive/MyDrive/TextPro2566/Surname


In [ ]:
from argparse import Namespace
import os
import json

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

In [ ]:
file_path = "surnames_with_splits.csv"
df = pd.read_csv(file_path)
(df.head())

,surname,nationality,split
0,Totah,Arabic,train
1,Abboud,Arabic,train
2,Fakhoury,Arabic,train
3,Srour,Arabic,train
4,Sayegh,Arabic,train


In [ ]:
import pandas as pd
surnames_path = '/content/drive/MyDrive/TextPro2566/Surname/surnames.csv'

#Read
surnames = pd.read_csv(surnames_path)
#Show head
surnames.head()

,surname,nationality,split
0,Gerges,Arabic,train
1,Nassar,Arabic,train
2,Kanaan,Arabic,train
3,Hadad,Arabic,train
4,Tannous,Arabic,train


## Vocabulary, Vectorizer, Dataset
we implement a dataset class, that returns the vectorized surname as well as the integer representing its nationality. Additionally returned is the length of the sequence, which is used in downstream computations to know where the final vector in the sequence is located. This is a part of the familiar sequence of steps - implementing Dataset, a Vectorizer, and a Vocabulary before the actual training can take place.

โค้ดที่ให้มีไว้เพื่อสร้างคลาส Vocabulary ซึ่งเป็นโครงสร้างพื้นฐานที่ใช้ในการจัดการกับโครงสร้างข้อมูลที่เกี่ยวข้องกับภาษา (text) ซึ่งประกอบไปด้วยฟังก์ชันสำคัญต่อไปนี้:

1. _init__: ฟังก์ชันนี้ถูกเรียกเมื่อสร้างวัตถุของคลาส Vocabulary โดยมีพารามิเตอร์

2. token_to_idx ซึ่งเป็นพารามิเตอร์ทางเลือก ถ้าไม่ได้รับค่า token_to_idx จะถือว่าเป็น None และจะกำหนดค่าให้เป็น dictionary ว่าง. และมี attribute _token_to_idx และ _idx_to_token ที่เก็บค่า mapping ระหว่าง token กับ index.

3. to_serializable: ฟังก์ชันนี้คืนค่า dictionary ที่เป็น serializable ซึ่งประกอบด้วย _token_to_idx.

4. from_serializable: เป็นเมทอดแบบ classmethod ที่ใช้ในการสร้างวัตถุ Vocabulary จาก dictionary ที่ serializable ที่รับมา.

5. add_token: เพิ่ม token เข้าไปใน vocabulary และคืนค่า index ที่เพิ่มเข้าไป.

6. add_many: เพิ่มลิสต์ของ tokens เข้าไปใน vocabulary และคืนค่าลิสต์ของ indices ที่เพิ่มเข้าไป.

7. lookup_token: ค้นหา index ที่เกี่ยวข้องกับ token ที่กำหนด และคืนค่า index.

8. lookup_index: ค้นหา token ที่เกี่ยวข้องกับ index ที่กำหนด และคืนค่า token.

__str__: คืนค่า string ที่บอกขนาดของ vocabulary.

__len__: คืนค่าจำนวนของ token ทั้งหมดใน vocabulary.

In [ ]:
from argparse import Namespace
import collections
import pandas as pd
import numpy as np

def split_data(data, train_proportion, val_proportion, test_proportion, seed):
    """Split data into train, validation, and test sets."""
    by_nationality = collections.defaultdict(list)

    for _, row in data.iterrows():
        by_nationality[row.nationality].append(row.to_dict())

    final_list = []
    np.random.seed(seed)

    for _, item_list in sorted(by_nationality.items()):
        np.random.shuffle(item_list)
        n = len(item_list)
        n_train = int(train_proportion * n)
        n_val = int(val_proportion * n)
        n_test = int(test_proportion * n)

        for item in item_list[:n_train]:
            item['split'] = 'train'
        for item in item_list[n_train:n_train + n_val]:
            item['split'] = 'val'
        for item in item_list[n_train + n_val:]:
            item['split'] = 'test'

        final_list.extend(item_list)

    final_data = pd.DataFrame(final_list)
    return final_data

def main():
    # Define arguments
    args = Namespace(
        raw_dataset_csv='/content/drive/MyDrive/TextPro2566/Surname/surnames.csv',
        train_proportion=0.7,
        val_proportion=0.15,
        test_proportion=0.15,
        output_munged_csv='/content/drive/MyDrive/TextPro2566/Surname/surnames.csv',
        seed=1337
    )

    # Load raw data
    surnames = pd.read_csv(args.raw_dataset_csv)

    # Split data
    final_surnames = split_data(surnames, args.train_proportion, args.val_proportion, args.test_proportion, args.seed)

    # Show the count of data after splitting
    print(final_surnames.split.value_counts())

    # Save the final data to a CSV file
    final_surnames.to_csv(args.output_munged_csv, index=False)

if __name__ == "__main__":
    main()

train    7773
test     1681
val      1659
Name: split, dtype: int64


1. สร้างฟังก์ชัน split_data เพื่อแยกตัวจัดการกับขั้นตอนการแบ่งข้อมูล
2. นำโค้ดเกี่ยวกับการแบ่งข้อมูลมาใส่ในฟังก์ชัน split_data
3. ใช้ฟังก์ชัน main เพื่อรวบรวมโค้ดและเรียกใช้ split_data
4. สร้างเงื่อนไข if __name__ == "__main__": เพื่อตรวจสอบว่าโค้ดถูกเรียกใช้โดยตรงหรือไม่
5. เพิ่มการแสดงผลจำนวนข้อมูลหลังแบ่งในฟังก์ชัน main
---



##Class Vocabulary

In [ ]:
class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token
                              for token, idx in self._token_to_idx.items()}

    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index

    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary

        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token

        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        """
        return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index

        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

The first stage in the vectorization pipeline is to map each character token in the surname to a unique integer. To accomplish this, we use the `SequenceVocabulary` data structure


##Class Sequence Vocabulary

คลาส SequenceVocabulary เป็นซับคลาสของคลาส Vocabulary และถูกออกแบบมาเพื่อให้รองรับการจัดการ sequence ของข้อความ. ลักษณะของคลาสนี้ได้รับการอธิบายได้ดังนี้:

1. __init__: ฟังก์ชันนี้ถูกใช้ในการสร้างวัตถุของคลาส SequenceVocabulary. โดยมีพารามิเตอร์เพิ่มเติมคือ unk_token, mask_token, begin_seq_token, และ end_seq_token ซึ่งเป็น default ค่าที่ถูกกำหนดไว้. ฟังก์ชันนี้เรียกฟังก์ชัน __init__ ของคลาสแม่ (super(SequenceVocabulary, self).__init__(token_to_idx)) เพื่อสร้าง dictionary สำหรับ mapping token กับ index. และกำหนดค่า index ของ mask_token, unk_token, begin_seq_token, และ end_seq_token โดยใช้เมทอด add_token ของคลาส Vocabulary.

2. to_serializable: เป็นฟังก์ชันที่คืนค่า dictionary ที่มีข้อมูลเกี่ยวกับคลาส SequenceVocabulary และที่สามารถนำไปใช้ในการ serialize object ของคลาสนี้ได้. ฟังก์ชันนี้ใช้ฟังก์ชัน to_serializable ของคลาสแม่เพื่อเก็บข้อมูลพื้นฐานของ vocabulary และเพิ่มข้อมูลเพิ่มเติมเกี่ยวกับ unk_token, mask_token, begin_seq_token, และ end_seq_token.

3. lookup_token: เป็นฟังก์ชันที่ใช้ในการค้นหา index ของ token ใน vocabulary. ถ้า token นั้นไม่มีอยู่ใน vocabulary จะให้ index ของ token UNK (unknown) แทน. การค้นหานี้ถือว่าสำคัญในกรณีที่ต้องการจัดการ tokens ที่ไม่ได้ปรากฎใน vocabulary และต้องการให้มีการแทนที่ด้วย token UNK. ในกรณีที่ unk_index มีค่าน้อยกว่า 0 (ไม่มีการใช้ UNK), ฟังก์ชันจะคืนค่า index ของ token หากมีใน vocabulary, แต่ถ้าไม่มีจะเกิด KeyError.

In [ ]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token
          or the UNK index if token isn't present.

        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
              for the UNK functionality
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

##Class SurnameVectorizer

The overall vectorizer is `SurnameVectorizer`, which populates the `SequenceVocabulary` by surname characters, and the normal `Vocabulary` for nationalities.

In [ ]:
class SurnameVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, char_vocab, nationality_vocab):
        """
        Args:
            char_vocab (Vocabulary): maps characters to integers
            nationality_vocab (Vocabulary): maps nationalities to integers
        """
        self.char_vocab = char_vocab
        self.nationality_vocab = nationality_vocab

    def vectorize(self, surname, vector_length=-1):
        """
        Args:
            title (str): the string of characters
            vector_length (int): an argument for forcing the length of index vector
        """
        indices = [self.char_vocab.begin_seq_index]
        indices.extend(self.char_vocab.lookup_token(token)
                       for token in surname)
        indices.append(self.char_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.char_vocab.mask_index

        return out_vector, len(indices)

    @classmethod
    def from_dataframe(cls, surname_df):
        """Instantiate the vectorizer from the dataset dataframe

        Args:
            surname_df (pandas.DataFrame): the surnames dataset
        Returns:
            an instance of the SurnameVectorizer
        """
        char_vocab = SequenceVocabulary()
        nationality_vocab = Vocabulary()

        for index, row in surname_df.iterrows():
            for char in row.surname:
                char_vocab.add_token(char)
            nationality_vocab.add_token(row.nationality)

        return cls(char_vocab, nationality_vocab)

    @classmethod
    def from_serializable(cls, contents):
        char_vocab = SequenceVocabulary.from_serializable(contents['char_vocab'])
        nat_vocab =  Vocabulary.from_serializable(contents['nationality_vocab'])

        return cls(char_vocab=char_vocab, nationality_vocab=nat_vocab)

    def to_serializable(self):
        return {'char_vocab': self.char_vocab.to_serializable(),
                'nationality_vocab': self.nationality_vocab.to_serializable()}


คลาส SurnameDataset เป็นคลาสที่เกี่ยวเนื่องมาจาก Dataset ใน PyTorch และถูกออกแบบมาเพื่อใช้ในการจัดการข้อมูลที่เกี่ยวข้องกับงานการทำนายนามสกุลและสัญชาติ (surname and nationality) ซึ่งมักจะมีลำดับ (sequence) ในข้อความ. ได้แก่ ข้อมูลเช่นนามสกุล, สัญชาติ, และการแบ่งชุด (split) ที่ระบุว่าเป็นชุด train, validation, หรือ test.

ตัวแปรหลัก:
1. surname_df: DataFrame ที่มีข้อมูลนามสกุลและสัญชาติทั้งหมด
2. _vectorizer: Vectorizer ที่ใช้ในการแปลงข้อมูลเป็นรูปแบบที่เหมาะสมสำหรับการนำเข้าโมเดล
3. _max_seq_length: ความยาวสูงสุดของ sequence ที่เก็บไว้ (จะมีการเพิ่ม 2 เพื่อให้มีพื้นที่สำหรับ token <BEGIN> และ <END>)
4. train_df, val_df, test_df: DataFrame ที่เก็บข้อมูลแบ่งตามชุด (train, validation, test)
5. train_size, validation_size, test_size: จำนวนข้อมูลในแต่ละชุด
6. _lookup_dict: Dictionary ที่เก็บ DataFrame และขนาดข้อมูลสำหรับแต่ละชุด
7. _target_split: ชื่อชุดที่ถูกเลือกในปัจจุบัน (default เป็น train)
8. _target_df, _target_size: DataFrame และขนาดข้อมูลสำหรับชุดที่ถูกเลือก
9. class_weights: น้ำหนักของแต่ละ class ในชุด train ที่ใช้ในการปรับสมดุลของข้อมูล (imbalance) ในกรณีทำนายแบบ multi-class classification

เมทอดหลัก:
1. set_split(split): เปลี่ยนชุดที่ถูกเลือกในปัจจุบันเป็นชุดที่ระบุ (train, validation, test)
2. __len__: คืนค่าขนาดของชุดที่ถูกเลือก
3. __getitem__: คืนค่าข้อมูลในรูปแบบที่เหมาะสมสำหรับนำเข้าโมเดล (features, labels, lengths)
4. get_num_batches(batch_size): คืนค่าจำนวน batch ที่เป็นไปได้ตามขนาด batch ที่ระบุ
5. generate_batches: เป็น generator function ที่ใช้ในการสร้าง batch ข้อมูลโดยใช้ PyTorch DataLoader และจัดการเลือก device ที่ถูกต้อง

เมทอดสร้างชุดข้อมูล:
1. load_dataset_and_make_vectorizer(surname_csv): โหลดชุดข้อมูลและสร้าง Vectorizer ใหม่จากข้อมูล training set
2. load_dataset_and_load_vectorizer(surname_csv, vectorizer_filepath): โหลดชุดข้อมูลและ Vectorizer ที่มีอยู่จาก

##Class SurnameDataset

In [ ]:
class SurnameDataset(Dataset):
    def __init__(self, surname_df, vectorizer):
        """
        Args:
            surname_df (pandas.DataFrame): the dataset
            vectorizer (SurnameVectorizer): vectorizer instatiated from dataset
        """
        self.surname_df = surname_df
        self._vectorizer = vectorizer

        self._max_seq_length = max(map(len, self.surname_df.surname)) + 2

        self.train_df = self.surname_df[self.surname_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.surname_df[self.surname_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.surname_df[self.surname_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

        # Class weights
        class_counts = self.train_df.nationality.value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.nationality_vocab.lookup_token(item[0])
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)


    @classmethod
    def load_dataset_and_make_vectorizer(cls, surname_csv):
        """Load dataset and make a new vectorizer from scratch

        Args:
            surname_csv (str): location of the dataset
        Returns:
            an instance of SurnameDataset
        """
        surname_df = pd.read_csv(surname_csv)
        train_surname_df = surname_df[surname_df.split=='train']
        return cls(surname_df, SurnameVectorizer.from_dataframe(train_surname_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, surname_csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer.
        Used in the case in the vectorizer has been cached for re-use

        Args:
            surname_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of SurnameDataset
        """
        surname_df = pd.read_csv(surname_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(surname_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file

        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of SurnameVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return SurnameVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json

        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets

        Args:
            index (int): the index to the data point
        Returns:
            a dictionary holding the data point's:
                features (x_data)
                label (y_target)
                feature length (x_length)
        """
        row = self._target_df.iloc[index]

        surname_vector, vec_length = \
            self._vectorizer.vectorize(row.surname, self._max_seq_length)

        nationality_index = \
            self._vectorizer.nationality_vocab.lookup_token(row.nationality)

        return {'x_data': surname_vector,
                'y_target': nationality_index,
                'x_length': vec_length}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset

        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size



def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## Model

The `SurnameClassifier` model is composed of an Embedding layer, the ElmanRNN, and a Linear layer.
The arguments to the model are

- the size of the embeddings - *hyperparameter*,
- the number of embeddings (i.e., vocabulary size) - determined by the data,
- the number of classes - determined by the data, and
- the hidden state size of the RNN - *hyperparameter*.

:::{important}
Although the two hyperparameters can take on any value, it is usually good to start with something small that ensures fast training, as a way verifying that the model actually works.
:::

ฟังก์ชั่น column_gather ใช้เพื่อดึงเวกเตอร์ที่กำหนดจากทุกๆ แถวในชุดข้อมูล y_out ตามค่าที่ระบุใน x_lengths ของแถวนั้นๆ:

Input:

y_out (torch.FloatTensor, torch.cuda.FloatTensor): เป็นเมทริกซ์ข้อมูลที่มีขนาด (batch, sequence, feature) ซึ่งเก็บข้อมูลที่โมเดลทำนาย
x_lengths (torch.LongTensor, torch.cuda.LongTensor): เป็นเวกเตอร์ขนาด (batch,) ที่เก็บความยาวของแต่ละ sequence ในชุดข้อมูล

Output:

y_out (torch.FloatTensor, torch.cuda.FloatTensor): เป็นเมทริกซ์ขนาด (batch, feature) ที่เก็บเวกเตอร์ที่ถูกดึงออกมาจากทุกๆ แถวใน y_out ตามค่าที่ระบุใน x_lengths

ขั้นตอนการทำงาน:

1. ทำการแปลง x_lengths จาก torch.LongTensor ให้เป็น numpy array และลบ 1 ออกจากค่าในแต่ละตำแหน่ง (x_lengths - 1)
2. สร้าง list ชื่อ out เพื่อเก็บเวกเตอร์ที่ดึงมาจากทุกๆ แถว
3. ในการวนลูปผ่านทุกแถวของชุดข้อมูล:
   *   นำ batch_index และ column_index ที่ได้จาก x_lengths มาใช้ในการดึงเวกเตอร์จาก y_out
  *   นำเวกเตอร์ที่ได้ไปเพิ่มใน list out
4. ใช้ torch.stack(out) เพื่อรวมเวกเตอร์ทั้งหมดใน list เป็นเมทริกซ์ y_out ขนาด (batch, feature) และคืนค่า

In [ ]:
def column_gather(y_out, x_lengths):
    '''Get a specific vector from each batch datapoint in `y_out`.

    More precisely, iterate over batch row indices, get the vector that's at
    the position indicated by the corresponding value in `x_lengths` at the row
    index.

    Args:
        y_out (torch.FloatTensor, torch.cuda.FloatTensor)
            shape: (batch, sequence, feature)
        x_lengths (torch.LongTensor, torch.cuda.LongTensor)
            shape: (batch,)

    Returns:
        y_out (torch.FloatTensor, torch.cuda.FloatTensor)
            shape: (batch, feature)
    '''
    x_lengths = x_lengths.long().detach().cpu().numpy() - 1

    out = []
    for batch_index, column_index in enumerate(x_lengths):
        out.append(y_out[batch_index, column_index])

    return torch.stack(out)


The ElmanRNN model

In [ ]:
class ElmanRNN(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=False):
        super(ElmanRNN, self).__init__()

        self.rnn_cell = nn.RNNCell(input_size, hidden_size)
        self.batch_first = batch_first
        self.hidden_size = hidden_size

    def _initial_hidden(self, batch_size):
        return torch.zeros((batch_size, self.hidden_size))

    def forward(self, x_in, initial_hidden=None):
        if self.batch_first:
            batch_size, seq_size, feat_size = x_in.size()
            x_in = x_in.permute(1, 0, 2)
        else:
            x_in = x_in.permute(0, 2, 1)
            seq_size, batch_size, feat_size = x_in.size()

        hiddens = []

        if initial_hidden is None:
            initial_hidden = self._initial_hidden(batch_size)
            initial_hidden = initial_hidden.to(x_in.device)

        hidden_t = initial_hidden

        for t in range(seq_size):
            hidden_t = self.rnn_cell(x_in[t], hidden_t)
            hiddens.append(hidden_t)

        hiddens = torch.stack(hiddens)

        if self.batch_first:
            hiddens = hiddens.permute(1, 0, 2)

        return hiddens


class SurnameClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings, num_classes,
                 rnn_hidden_size, batch_first=True, padding_idx=0):
        super(SurnameClassifier, self).__init__()

        self.emb = nn.Embedding(num_embeddings=num_embeddings,
                                embedding_dim=embedding_size,
                                padding_idx=padding_idx)
        self.rnn = ElmanRNN(input_size=embedding_size,
                            hidden_size=rnn_hidden_size,
                            batch_first=batch_first)
        self.fc1 = nn.Linear(in_features=rnn_hidden_size,
                             out_features=rnn_hidden_size)
        self.fc2 = nn.Linear(in_features=rnn_hidden_size,
                             out_features=num_classes)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x_in, x_lengths=None, apply_softmax=False):
        x_embedded = self.emb(x_in)
        y_out = self.rnn(x_embedded)

        if x_lengths is not None:
            y_out = column_gather(y_out, x_lengths)
        else:
            y_out = y_out[:, -1, :]

        y_out = F.relu(self.fc1(self.dropout(y_out)))
        y_out = self.fc2(self.dropout(y_out))

        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)

        return y_out

คลาส ElmanRNN คือโมเดลที่ใช้ RNNCell ของ PyTorch ในการสร้าง Elman RNN โดยโมเดล Elman RNN ที่ใช้ RNNCell ในการอัปเดต hidden state ของ sequence

ส่วนสำคัญ:

1. __init__: สร้างโมเดล Elman RNN โดยรับ input size, hidden size, และ batch_first ในการกำหนดว่า batch dimension จะอยู่ที่มุมแรกหรือไม่.
2. _initial_hidden: สร้าง hidden state เริ่มต้นที่มีค่าเป็นศูนย์ ขนาด (batch_size, hidden_size).
3. forward: ทำ forward pass โดยรับ input tensor x_in และ hidden state เริ่มต้น (ถ้าไม่ระบุจะใช้ hidden state 0) และคืน hidden states ที่ถูกสร้างขึ้นในแต่ละ time step ของ sequence.

คลาส SurnameClassifier คือโมเดลที่ใช้ Elman RNN และ MLP (Multi-Layer Perceptron) ในการจัดการงานการทำนาย: เป็นโมเดลที่ใช้ Elman RNN ในการดึง feature และ MLP ในการทำนายสัญชาติของนามสกุล

ส่วนสำคัญ:

1. __init__: สร้างโมเดล SurnameClassifier โดยรับพารามิเตอร์ที่จำเป็นเพื่อกำหนดคุณสมบัติของโมเดล เช่น embedding size, num_embeddings, num_classes, และอื่นๆ.
2. forward: ทำ forward pass โดยรับ input tensor x_in, embeddings โดยใช้ embedding layer (self.emb), ทำ forward pass กับ Elman RNN (self.rnn), และผ่าน MLP (self.fc1 และ self.fc2) เพื่อทำนายคลาส. สามารถใส่ความยาวของแต่ละ sequence (x_lengths) เพื่อให้ได้เวกเตอร์ที่อยู่ที่ตำแหน่งสุดท้ายของแต่ละ sequence.
3. apply_softmax: ถ้าตั้งค่าเป็น True จะใช้ softmax activation function สำหรับการทำนาย output. ในกรณีใช้กับ Cross Entropy losses ควรตั้งค่าเป็น False


In [ ]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

set_seed_everywhere คือฟังก์ชันที่ใช้ในการตั้งค่า seed สำหรับการสร้างตัวเลขสุ่มใน NumPy และ PyTorch ทำให้การสร้างตัวเลขสุ่มเป็นไปตามลำดับที่กำหนดโดย seed เดียวกัน
ส่วนสำคัญ:

1. seed : คือค่า seed ที่ถูกใช้ในการกำหนดการสร้างตัวเลขสุ่ม การกำหนด seed เดียวกันจะทำให้ได้ผลลัพธ์ที่เหมือนกันทุกรอบ.
2. cuda: คือตัวแปรที่บอกว่าจะใช้ CUDA (GPU) หรือไม่ ถ้าใช้ CUDA จะต้องตั้งค่า seed สำหรับ GPU ด้วย.
3. handle_dirs คือฟังก์ชันที่ใช้ในการตรวจสอบและสร้าง directory หาก directory นั้นยังไม่มีอยู่ ซึ่งมักใช้ในกรณีที่ต้องการบันทึกโมเดลหรือไฟล์อื่นๆ ที่ต้องการเก็บไว้ใน directory ที่กำหนด.

## Settings

โค้ดนี้มีวัตถุประสงค์เพื่อกำหนดค่าพารามิเตอร์สำหรับการฝึกโมเดลและทำงานกับข้อมูลในแบบคำสั่งเฉพาะของ Python โดยใช้ argparse.Namespace เป็นโครงสร้างข้อมูล. ดูดวงเพิ่มเติมข้างล่าง:

Data and path information:
- surname_csv: ชื่อไฟล์ CSV ที่มีข้อมูลนามสกุลและข้อมูลการแบ่งกลุ่ม (splits).
- vectorizer_file: ชื่อไฟล์ JSON ที่จะใช้เก็บข้อมูล vectorizer.
- model_state_file: ชื่อไฟล์เพื่อบันทึก state ของโมเดล (weights).
- save_dir: ที่เก็บไฟล์ที่จะบันทึก

Model hyperparameters:
- char_embedding_size: ขนาดของ embedding สำหรับตัวอักษร.
- rnn_hidden_size: ขนาดของ hidden state ในโมเดล RNN.

Training hyperparameters:
- num_epochs: จำนวน epoch ที่จะทำการฝึก.
- learning_rate: อัตราการเรียนรู้ของโมเดล.
- batch_size: ขนาดของ batch ในการฝึก.
- seed: seed สำหรับการสุ่ม (random seed).
- early_stopping_criteria: เกณฑ์ในการหยุดการฝึกเร็ว (early stopping criteria)

Runtime hyperparameters:
- cuda: ใช้ GPU (True) หรือไม่ (False)
- reload_from_files: โหลดข้อมูลจากไฟล์หรือไม่.
- expand_filepaths_to_save_dir: สร้าง path สำหรับบันทึกข้อมูลใน save_dir

โค้ดทำการตรวจสอบว่า CUDA (GPU) ใช้ได้หรือไม่ และกำหนด args.device ตามนั้น โดยใช้ GPU ถ้า args.cuda เป็น True และใช้ CPU ถ้า args.cuda เป็น False. สุดท้ายก็ทำการตั้งค่า seed สำหรับการสุ่มและสร้าง directory ที่จะใช้ในการบันทึกไฟล์ผลลัพธ์.

สรุปได้ว่าโค้ดนี้เป็นการกำหนดค่าพารามิเตอร์และทำการเตรียมสภาพแวดล้อมสำหรับการฝึกโมเดลในโครงการที่ใช้งาน Python.

In [ ]:
args = Namespace(
    # Data and path information
    surname_csv="surnames_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/ch6/surname_classification",

    # Model hyper parameter
    char_embedding_size=200,
    rnn_hidden_size=900,

    # Training hyper parameter
    num_epochs=30,
    learning_rate=1e-4,
    batch_size=64,
    seed=1337,
    early_stopping_criteria=5,

    # Runtime hyper parameter
    cuda=True,
    catch_keyboard_interrupt=True,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
)

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")

print("Using CUDA: {}".format(args.cuda))


if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)

Using CUDA: False


In [ ]:
if args.reload_from_files and os.path.exists(args.vectorizer_file):
    # training from a checkpoint
    dataset = SurnameDataset.load_dataset_and_load_vectorizer(args.surname_csv,
                                                              args.vectorizer_file)
else:
    # create dataset and vectorizer
    dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
    dataset.save_vectorizer(args.vectorizer_file)

vectorizer = dataset.get_vectorizer()

classifier = SurnameClassifier(embedding_size=args.char_embedding_size,
                               num_embeddings=len(vectorizer.char_vocab),
                               num_classes=len(vectorizer.nationality_vocab),
                               rnn_hidden_size=args.rnn_hidden_size,
                               padding_idx=vectorizer.char_vocab.mask_index)

In [ ]:
print(vectorizer.char_vocab)
print(dataset._max_seq_length)
print(vectorizer.nationality_vocab)

<Vocabulary(size=81)>
20
<Vocabulary(size=19)>


In [ ]:
dataset[0]

{'x_data': array([2, 4, 5, 6, 7, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'y_target': 0,
 'x_length': 7}

In [ ]:
dataset._target_df.iloc[0]

surname         Totah
nationality    Arabic
split           train
Name: 0, dtype: object

In [ ]:
print([vectorizer.char_vocab.lookup_index(i) for i in dataset[0]['x_data']])

['<BEGIN>', 'T', 'o', 't', 'a', 'h', '<END>', '<MASK>', '<MASK>', '<MASK>', '<MASK>', '<MASK>', '<MASK>', '<MASK>', '<MASK>', '<MASK>', '<MASK>', '<MASK>', '<MASK>', '<MASK>']


In [ ]:
classifier

SurnameClassifier(
  (emb): Embedding(81, 200, padding_idx=0)
  (rnn): ElmanRNN(
    (rnn_cell): RNNCell(200, 900)
  )
  (fc1): Linear(in_features=900, out_features=900, bias=True)
  (fc2): Linear(in_features=900, out_features=19, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
# @title
x_in = torch.LongTensor(dataset[0]['x_data']).unsqueeze(0)
x_len = torch.LongTensor([dataset[0]['x_length']])
print(x_in.shape)
print(x_len.shape)
classifier(x_in, x_len).shape

torch.Size([1, 20])
torch.Size([1])


torch.Size([1, 19])

In [ ]:
dataset[0]['x_data']
x_in = torch.LongTensor(dataset[0]['x_data']).unsqueeze(0)
out_emb = classifier.emb(x_in)
out_emb.shape

torch.Size([1, 20, 200])

In [ ]:
out_rnn = classifier.rnn(out_emb)
out_rnn.shape

torch.Size([1, 20, 900])

In [ ]:
out_fc1 = F.relu(classifier.fc1(F.dropout(out_rnn, 0.5)))
out_fc1.shape

torch.Size([1, 20, 900])

In [ ]:
out_fc2 = classifier.fc2(F.dropout(out_fc1, 0.5))
out_fc2.shape

torch.Size([1, 20, 19])

In [ ]:
F.softmax(out_fc2, dim=1).shape

torch.Size([1, 20, 19])

## Training Routine

The training routine follows the standard formula. For a single batch of data, apply the model and compute the prediction vectors. Use the `CrossEntropyLoss()` loss function and the ground truth to compute a loss value. Using the loss value and an optimizer, compute the gradients and update the weights of the model using those gradients. Repeat this for each batch in the training data. Proceed similarly with the validation data, but set the model in eval mode so as to prevent backpropagating. Instead, the validation data is used only to give a less­biased sense of how the model is performing. Repeat this routine for a specific number of epochs or a stopping condition (e.g. loss is less than a threshold, or loss stoped changing in the most recent two epochs) is met.

In [ ]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}


def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= loss_tm1:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state


def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

โค้ดนี้เป็นส่วนหนึ่งของการทำงานในกระบวนการฝึกโมเดล (training loop) และการตรวจสอบประสิทธิภาพของโมเดลในช่วงการฝึก มีฟังก์ชันหลาย ๆ ฟังก์ชันที่ทำหน้าที่ต่าง ๆ ดังนี้:

1. make_train_state(args): ฟังก์ชันนี้ใช้สร้าง dictionary เริ่มต้นที่จะใช้เก็บสถานะของการฝึกโมเดล โดยกำหนดค่าเริ่มต้นที่สำคัญ เช่น stop_early เป็น False, early_stopping_step เป็น 0, early_stopping_best_val เป็น infinity, learning_rate เป็นค่าที่กำหนดใน args.learning_rate, epoch_index เริ่มต้นที่ 0, และกำหนด list ว่างสำหรับเก็บค่า loss และ accuracy ของ training และ validation.

2. update_train_state(args, model, train_state): ฟังก์ชันนี้ใช้ปรับปรุงสถานะการฝึกโมเดลตามความคืบหน้าของการฝึก โดยรับเข้ามาตัวแปร args สำหรับการตั้งค่าหลัก, model ที่กำลังถูกฝึก, และ train_state ที่เก็บสถานะของการฝึกโมเดล. ฟังก์ชันนี้ทำหลายงาน ได้แก่:
 - บันทึกโมเดลไว้เป็น checkpoint ที่ epoch แรก.
 - บันทึกโมเดลที่ดีที่สุดถ้ามีประสิทธิภาพดีกว่าของ checkpoint ก่อนหน้า.
 - ทำการ early stopping หากไม่มีการปรับปรุงประสิทธิภาพ (ลด loss) ใน args.early_stopping_criteria epochs ล่าสุด.
3. compute_accuracy(y_pred, y_target): ฟังก์ชันนี้ใช้คำนวณความแม่นยำของโมเดล โดยรับค่า y_pred ที่เป็นผลลัพธ์ที่โมเดลทำนายและ y_target ที่เป็นคำตอบที่ถูกต้อง. ความแม่นยำถูกนับและคืนค่าเป็นเปอร์เซ็นต์ของตัวอย่างที่ถูกต้อง.

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# ...

# Define the classifier, optimizer, loss function, and scheduler
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)

loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                  mode='min', factor=0.5,
                                                  patience=1)

# Initialize training state
train_state = make_train_state(args)

# Setup progress bars
epoch_bar = tqdm(desc='training routine',
                 total=args.num_epochs,
                 position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train',
                 total=dataset.get_num_batches(args.batch_size),
                 position=1,
                 leave=True)

dataset.set_split('val')
val_bar = tqdm(desc='split=val',
               total=dataset.get_num_batches(args.batch_size),
               position=1,
               leave=True)

try:
    # Training loop
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Training phase
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            optimizer.zero_grad()

            # Forward pass
            y_pred = classifier(x_in=batch_dict['x_data'], x_lengths=batch_dict['x_length'])

            # Compute loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            running_loss += (loss.item() - running_loss) / (batch_index + 1)

            # Backward pass
            loss.backward()
            optimizer.step()

            # Compute accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            train_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Validation phase
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.eval()

        with torch.no_grad():
            for batch_index, batch_dict in enumerate(batch_generator):
                y_pred = classifier(x_in=batch_dict['x_data'], x_lengths=batch_dict['x_length'])

                # Compute loss
                loss = loss_func(y_pred, batch_dict['y_target'])
                running_loss += (loss.item() - running_loss) / (batch_index + 1)

                # Compute accuracy
                acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
                running_acc += (acc_t - running_acc) / (batch_index + 1)

                val_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
                val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        # Update training state
        train_state = update_train_state(args=args, model=classifier, train_state=train_state)

        # Adjust learning rate based on validation loss
        scheduler.step(train_state['val_loss'][-1])

        # Update progress bars
        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()

        # Check for early stopping
        if train_state['stop_early']:
            break

except KeyboardInterrupt:
    print("Exiting loop")

# Save the trained model
torch.save(classifier.state_dict(), 'classifier_checkpoint.pth')

training routine:   0%|          | 0/30 [00:00<?, ?it/s]

split=train:   0%|          | 0/121 [00:00<?, ?it/s]

split=val:   0%|          | 0/25 [00:00<?, ?it/s]

โค้ดนี้เป็นส่วนหลักของกระบวนการฝึกโมเดล (training loop) และการตรวจสอบประสิทธิภาพของโมเดลในช่วงการฝึกโดยใช้ PyTorch โดยทำหน้าที่ต่าง ๆ ดังนี้:

- classifier = classifier.to(args.device): เปลี่ยนโมเดลและพารามิเตอร์ของโมเดลเพื่อให้ทำงานบน GPU (หากใช้ CUDA) หรือ CPU (หากไม่ใช้ CUDA) ตาม args.device.

- dataset.class_weights = dataset.class_weights.to(args.device): เปลี่ยน class weights ของ dataset เพื่อให้ทำงานบน GPU (หากใช้ CUDA) หรือ CPU (หากไม่ใช้ CUDA) ตาม args.device.

- loss_func = nn.CrossEntropyLoss(dataset.class_weights): กำหนด loss function ในรูปแบบ Cross Entropy Loss และใช้ class weights ที่ถูกกำหนดไว้ใน dataset.

- optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate): กำหนด optimizer ในรูปแบบ Adam สำหรับการปรับปรุงพารามิเตอร์ของโมเดล โดยใช้ learning rate จาก args.learning_rate.

- scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=1): กำหนด learning rate scheduler ในรูปแบบ ReduceLROnPlateau ซึ่งจะลด learning rate ทุกรอบที่ validation loss ไม่ลดลง (mode='min') ด้วย factor=0.5 และ patience=1.

- train_state = make_train_state(args): สร้าง dictionary เริ่มต้นสำหรับเก็บสถานะของการฝึกโมเดลโดยใช้ฟังก์ชัน make_train_state ที่กำหนดค่าเริ่มต้นของสถานะ.

- epoch_bar = tqdm(...), train_bar = tqdm(...), val_bar = tqdm(...): กำหนด progress bars สำหรับแสดงความคืบหน้าในการฝึกโมเดล.

ลูปฝึกโมเดล:

  - ในลูปนี้มีการตั้งค่าต่าง ๆ เช่นตัวแปร train_state['epoch_index'], dataset.set_split(), batch_generator, running_loss, และ running_acc.
  - ลูปฝึกโมเดลและปรับปรุงพารามิเตอร์โดยใช้ backpropagation.
  - คำนวณค่า loss และ accuracy ของชุดข้อมูลฝึก
  - แสดงความคืบหน้าของการฝึกโมเดลผ่าน progress bar.

ลูปทำการทดสอบบนชุดข้อมูล validation:

  - ในลูปนี้มีการตั้งค่าต่าง ๆ เช่นตัวแปร dataset.set_split(), batch_generator, running_loss, และ running_acc.
  - คำนวณค่า loss และ accuracy ของชุดข้อมูล validation.
  - แสดงความคืบหน้าของการทดสอบ validation ผ่าน progress bar.
  - train_state = update_train_state(...): ปรับปรุงสถานะการฝึกโมเดลบันทึกโมเดลที่ดีที่สุด, และทำ early stopping หากไม่มีการปรับปรุงตามเงื่อนไขที่กำหนด.

- scheduler.step(train_state['val_loss'][-1]): ปรับ learning rate ตามค่า validation loss ล่าสุด.

- ปรับค่า progress bar และบันทึกค่าสถานะทั้งหมด.

หลุดจากลูปฝึกโมเดลหากมีการกด Ctrl+C จากคีย์บอร์ด.

In [ ]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset,
                                   batch_size=args.batch_size,
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_data'],
                         x_lengths=batch_dict['x_length'])

    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [ ]:
print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

Test loss: 1.456196395250467;
Test Accuracy: 58.413461538461554


### Inference

In [ ]:
def predict_nationality(surname, classifier, vectorizer):
    vectorized_surname, vec_length = vectorizer.vectorize(surname)
    vectorized_surname = torch.tensor(vectorized_surname).unsqueeze(dim=0)
    vec_length = torch.tensor([vec_length], dtype=torch.int64)

    result = classifier(vectorized_surname, vec_length, apply_softmax=True)
    probability_values, indices = result.max(dim=1)

    index = indices.item()
    prob_value = probability_values.item()

    predicted_nationality = vectorizer.nationality_vocab.lookup_index(index)

    return {'nationality': predicted_nationality, 'probability': prob_value, 'surname': surname}

In [ ]:
# surname = input("Enter a surname: ")
classifier = classifier.to("cpu")
for surname in ['Rueksen', 'Koonvisal']:
    print(predict_nationality(surname, classifier, vectorizer))

{'nationality': 'Dutch', 'probability': 0.8372834324836731, 'surname': 'Rueksen'}
{'nationality': 'Thai', 'probability': 0.7013441324234009, 'surname': 'Koonvisal'}


# Name : Sukanya Rueksen

# ID : 66076045